In [89]:
#importing data
import csv
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
import DB

2017-04-07 13:49:23,429 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-04-07 13:49:23,429 INFO sqlalchemy.engine.base.Engine ()
2017-04-07 13:49:23,429 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-04-07 13:49:23,443 INFO sqlalchemy.engine.base.Engine ()
2017-04-07 13:49:23,444 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("access_log")
2017-04-07 13:49:23,445 INFO sqlalchemy.engine.base.Engine ()
2017-04-07 13:49:23,447 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("account_company")
2017-04-07 13:49:23,448 INFO sqlalchemy.engine.base.Engine ()
2017-04-07 13:49:23,449 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("account_external")
2017-04-07 13:49:23,450 INFO sqlalchemy.engine.base.Engine ()
2017-04-07 13:49:23,451 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("authorisation_level")
2017-04-07 13:49:23,451 INFO sqlalchemy.engine.base.Engine ()
2017-0

In [47]:
access_log = "access_log.csv"

files = ['employee_department.csv',
'access_log.csv'  ,           'invoice_purchase_order.csv',
'account_company.csv',        'invoice_transactions.csv',
'account_external.csv',       'public_holidays_zrh.csv',
'authorization_level.csv',    'system_users.csv',
'bank_transfer_file.csv',     'transactions.csv',
'calendar.csv',               'user_groups.csv',
'department.csv',             'vendor.csv',
'employee.csv',               'vendor_address.csv']

with open()

employee_department_data = pd.read_csv('employee_department.csv', sep='|')
access_log_data = pd.read_csv('access_log.csv', sep='|')
invoice_purchase_order_data = pd.read_csv('invoice_purchase_order.csv', sep='|')
account_company_data = pd.read_csv('account_company.csv', sep='|')
invoice_transactions_data = pd.read_csv('invoice_transactions.csv', sep='|')
account_external_data = pd.read_csv('account_external.csv', sep='|')
public_holidays_zrh_data = pd.read_csv('public_holidays_zrh.csv', sep='|')
authorization_level_data = pd.read_csv('authorization_level.csv', sep='|')
system_users_data = pd.read_csv('system_users.csv', sep='|')
transactions_data = pd.read_csv('transactions.csv', sep='|')
calendar_data = pd.read_csv('calendar.csv', sep='|')
user_groups_data = pd.read_csv('user_groups.csv', sep='|')
department_data = pd.read_csv('department.csv', sep='|')   
vendor_data = pd.read_csv('vendor.csv', sep='|')
employee_data = pd.read_csv('employee.csv', sep='|')
vendor_address_data = pd.read_csv('vendor_address.csv', sep='|')
#bank_transfer_file_data = pd.read_csv('bank_transfer_file.csv') 



In [73]:
#Get # vendors with incomplete details.

vendors = []
vendor_addr = []
addr_header = None

with open('data/vendor.csv', "r") as input_file:
    reader = csv.reader(input_file, delimiter="|")
    next(reader) #skip header
    vendors = [row for row in reader]

    
with open('data/vendor_address.csv', "r") as input_file:
    reader = csv.reader(input_file, delimiter="|")
    addr_header = next(reader) #skip header
    vendor_addr = [row for row in reader]

counter = 0
vendor_ids = set()
inc_vendors = []

for idx,v in enumerate(vendors):
    for entry in v:
        #print(entry)
        if not entry:
            counter += 1
            vendor_ids.add(v[0])
            inc_vendors.append(v)
            break
            
print(counter)

for idx,v in enumerate(vendor_addr):
    for entry in v:
        if not entry and v[0] not in vendor_ids:
            counter += 1
            v_2 = []
            for e in v:
                if not e:
                    v_2.append("EMPTY")
                else:
                    v_2.append(e)
                    
            inc_vendors.append(v_2)
            vendor_ids.add(v[0])
            break
            
print(counter)
print(vendor_ids)

with open('analysis/incomplete_vendors.csv','w') as out_file:
    out_file.write("\n"+",".join(addr_header))
    for v in inc_vendors:
        out_file.write("\n"+",".join(v))

0
10
{'VDA-00773', 'VDA-00449', 'VDA-00323', 'VDA-00772', 'VDA-00398', 'VDA-00206', 'VDA-00109', 'VDA-00547', 'VDA-00597', 'VDA-00346'}


In [86]:
#Counting split payments
transactions = []
trans_header = []

with open('data/transactions.csv', "r") as input_file:
    reader = csv.reader(input_file, delimiter="|")
    trans_header = next(reader) #skip header
    transactions = [row for row in reader]

payments = {}
split_counts = 0
split_payments = []

for idx, row in enumerate(transactions):
    invoice_note = row[7]
    if invoice_note not in payments:
        payments[invoice_note] = [row]
    else:
        payments[invoice_note].append(row)

for key,val in payments.items():
    if len(val) > 1:
        split_counts += len(val)
        split_payments.extend(val)

with open('analysis/split_payments.csv','w') as out:
    out.write(",".join(trans_header)+"\n")
    for t in split_payments:
        out.write(",".join(t)+"\n")

In [143]:
#Consecutive input ids

def generate_next_ID(current_ID):
    number = int(current_ID.split("-")[1])
    next_number = str(number+1).zfill(9) #Padding as ID length is 9.
    return "PUO-"+next_number


def generate_next_tampered_ID(current_ID):
    number = int(current_ID.split("-")[1])
    next_number = str(number+1) #No padding even though ID length is 9.
    return "PUO-"+next_number

def generate_previous_ID(current_ID):
    number = int(current_ID.split("-")[1])
    next_number = str(number-1).zfill(9)
    return "PUO-"+next_number


def generate_previous_tampered_ID(current_ID):
    number = int(current_ID.split("-")[1])
    next_number = str(number-1)
    return "PUO-"+next_number

S = DB.sess
query = S.query(DB.InvoicePurchaseOrder)
purchase_IDs = {}

for row in query:
    if row.po_id in purchase_IDs:
        print("RAAAGE")
    purchase_IDs[row.po_id] = False

consecutives = 0
consec = set()

for po_ID in purchase_IDs:
    next_ID = generate_next_ID(po_ID)
    previous_ID = generate_previous_ID(po_ID)
    next_tampered_ID = generate_next_tampered_ID(po_ID)
    previous_tampered_ID = generate_previous_tampered_ID(po_ID)
    
    if next_ID in purchase_IDs:# and previous_ID in purchase_IDs:
#        if not purchase_IDs[po_ID]:
#            purchase_IDs[po_ID] = True
#            consecutives += 1
#            consec.add(po_ID)
            
#        if not purchase_IDs[next_ID]:
#            purchase_IDs[next_ID] = True
#            consecutives += 1
#            consec.add(next_ID)
        
        consec.add(next_ID)
        consec.add(po_ID)
        #consec.add(previous_ID)
    
    if next_tampered_ID in purchase_IDs:# and previous_tampered_ID in purchase_IDs:
        consec.add(next_tampered_ID)
        consec.add(po_ID)
        #consec.add(previous_tampered_ID)
        #consec.add(previous_ID)
#    if previous_ID in purchase_IDs:
#        if not purchase_IDs[po_ID]:
#            purchase_IDs[po_ID] = True
#            consecutives += 1
#        consec.add(po_ID)
            
#        if not purchase_IDs[previous_ID]:
#            purchase_IDs[previous_ID] = True
#            consecutives += 1
#        consec.add(previous_ID)
            
#print(len(purchase_IDs))
print(consecutives)
print(sorted(consec))
print(len(consec))


2017-04-07 15:51:41,778 INFO sqlalchemy.engine.base.Engine SELECT invoice_purchase_order.invoice_id AS invoice_purchase_order_invoice_id, invoice_purchase_order.po_id AS invoice_purchase_order_po_id, invoice_purchase_order.vendor_id AS invoice_purchase_order_vendor_id, invoice_purchase_order.invoice_dt AS invoice_purchase_order_invoice_dt, invoice_purchase_order.invoice_image_path AS invoice_purchase_order_invoice_image_path, invoice_purchase_order.payment_due_by_dt AS invoice_purchase_order_payment_due_by_dt, invoice_purchase_order.received_dt AS invoice_purchase_order_received_dt, invoice_purchase_order.received_by_user_id AS invoice_purchase_order_received_by_user_id, invoice_purchase_order.total AS invoice_purchase_order_total 
FROM invoice_purchase_order
2017-04-07 15:51:41,779 INFO sqlalchemy.engine.base.Engine ()
0
['PUO-001546992', 'PUO-001546993', 'PUO-001546994', 'PUO-001546995', 'PUO-001546996', 'PUO-001546997', 'PUO-001546998', 'PUO-001546999', 'PUO-001547000', 'PUO-0015470

In [146]:
def generate_next_ID(current_ID):
    return current_ID + 1


S = DB.sess
query = S.query(DB.InvoicePurchaseOrder)
purchase_IDs = {}

for row in query:
    raw_ID = int(row.po_id.split("-")[1])
    purchase_IDs[raw_ID] = False

consecutives = 0
consec = set()

for po_ID in purchase_IDs:
    next_ID = po_ID + 1
    #revious_ID = po_ID -1
    
    if next_ID in purchase_IDs:#and previous_ID in purchase_IDs:
        consec.add(po_ID)
        consec.add(next_ID)
        #onsec.add(previous_ID)
    
print(sorted(consec))
print(len(consec))

2017-04-07 16:15:37,936 INFO sqlalchemy.engine.base.Engine SELECT invoice_purchase_order.invoice_id AS invoice_purchase_order_invoice_id, invoice_purchase_order.po_id AS invoice_purchase_order_po_id, invoice_purchase_order.vendor_id AS invoice_purchase_order_vendor_id, invoice_purchase_order.invoice_dt AS invoice_purchase_order_invoice_dt, invoice_purchase_order.invoice_image_path AS invoice_purchase_order_invoice_image_path, invoice_purchase_order.payment_due_by_dt AS invoice_purchase_order_payment_due_by_dt, invoice_purchase_order.received_dt AS invoice_purchase_order_received_dt, invoice_purchase_order.received_by_user_id AS invoice_purchase_order_received_by_user_id, invoice_purchase_order.total AS invoice_purchase_order_total 
FROM invoice_purchase_order
2017-04-07 16:15:37,937 INFO sqlalchemy.engine.base.Engine ()
[709771, 709772, 709773, 709774, 709775, 709776, 709777, 709778, 709779, 709780, 709781, 709782, 709783, 709784, 709785, 709786, 709787, 709788, 709789, 709790, 709791,

In [130]:
print("PUO-001887914" in consec)
print("PUO-001546994" in consec)

True
True


In [ ]:
#Amount stolen